# NLP Final Project
R. Mattson


Tuesday, Dec. 14 2021

<!-- TODO:

Are my binary rules well formed? 

Does chart build up to Z sufficiently? I don't think so...?

does it propagate well...

What is the sum of other things??? 

WE should only care about S on the 

??LIKELIHOOD - review probability shit and put it on a slide

I want 

rule building -> chart building -> inside-outside indices ->  E step gives likelihood -> whats sum of other counts? -> M step correctly calculates thetas -> em exits in a timely manner -> save g, its the core of PCFG.

I think I could use a heuristic for avg_expected_counts... but if its not likelihood, we will run into problems later... 
e_step(sents, rules, G_i)
and we want vector as long as our authors, and we say pred_i  = argmax(vector)

Choose to progress on this or do presentation

Goals:

- make presentation slides
- get inside-outside working well enough to get model trained for each author
- compare likelihood of expectation step on test data
- compare PCFG prediction performance to some simple models, eg naive bayes + BOW


 -->


In [ ]:
import torch
import numpy as np
import itertools
from tqdm.notebook import tqdm
import random as rand

# LOAD INITIAL RULES AND WORDs

In [ ]:
def load_text(path_to_text):
    
    words = np.loadtxt(path_to_text, delimiter="/n", dtype=str,)
    
    sents = [[sent.split()] for sent in words]
    sents = [[sent] for sent in sents if len(sent) > 2] # just a check
   
    return words
rules_d = {}

In [ ]:
def load_rules_from_pcfg(path_to_pcfg):
    rules = np.loadtxt(path_to_pcfg, delimiter="/n", dtype=str)
    binary_rule_length = len(rules[0].split())
    rules = np.unique(rules)

    binary_rules = []
    unary_rules = []
    nts = []
    
    for rule in rules:
        rule = rule.split()
        
        if rule[0] not in nts:
            nts.append(rule[0])
        if len(rule) == binary_rule_length:
            # print(rule)
            prob = rand.random()
            rule = np.array([rule[0], rule[2], rule[3], prob])
            binary_rules.append(rule)

        elif len(rule) < binary_rule_length:
            prob = rand.random()
            rule = np.array([rule[0], rule[2], prob])
            unary_rules.append(rule)
        else:
            print("error")
            print(rule)
    

    
    binary_rules = np.array(binary_rules) 
    unary_rules = np.array(unary_rules) 
    return unary_rules, binary_rules, nts
    
unary_rules, binary_rules, nts = load_rules_from_pcfg("../make-data/pcfg_ug.txt")
# binary_rules

# Inside-Outside Algorithm

In [ ]:
class Chart():
    def __init__(self, nts, nts2idx, unary_rules, binary_rules, words):
        self.n = len(words)
        self.v = len(nts)
        self.nts = nts
        self.words = words
        self.nts2idx = nts2idx
        # self.nts2idx = nts2idx
        self.arr = np.zeros([self.n, self.n +1, self.v])


    def set(self, i, j, A, value):
        rule_idx = self.nts2idx[A]
        self.arr[i, j, rule_idx] = value
        return

    def get(self, i, j, A):
        rule_idx = self.nts2idx[A]
        return self.arr[i, j, rule_idx]
  

    def add(self, i, j, A, value):
        rule_idx = self.nts2idx[A]
        self.arr[i, j, rule_idx] += value
        return 

    def print(self, num_slices=1):
        print("shape:", self.arr.shape)
        if num_slices <= 0:
            num_slices = self.v
        # for r in range(num_slices):
            # print(self.nts[r])
        for i in range(self.n):
            for j in range(self.n+1):
                if i == j:
                    # print(self.words[i], end="| ")
                    print('{:.6s}'.format(self.words[i]), end="| ")
                    #  print(self.words[i].center(4), end= "| ")
                else:
                    # print('{:.6f}'.format(self.arr[i, j, r]), end="| ")
                    assert len(self.arr[i, j]) == self.v
                    max_value = np.max(self.arr[i, j])
                    if max_value > 0.0:
                        print('{:.6f}'.format(max_value), end="| ")
                    else: 
                        print("000000", end="| ")
            print()

    

# lmao


In [ ]:
# def inside(sent, params=[]):
def inside(words, unary_rules, binary_rules, nts, nts2idx, g):
    # print("ENTERING INSIDE")
    # print(type(unary_rules), type(binary_rules))

    chart = []
    # words = sent.split(" ")
    # print(words)
    n = len(words)
    num_unary_rules_used = 0
    chart = Chart(nts, nts2idx, unary_rules, binary_rules, words)

    # TODO- where is good?
    # print("pretty" in words)
    unary = [x for x in unary_rules if x[1] in words]

    v = max(len(unary), len(binary_rules))
    substitute_rule = rand.choice(unary_rules)

    for k in range(n):
        # print("word", words[k])
        relevant_rules = [x for x in unary if words[k] in x]
        # print(rules)
        # print("Chart span", k, k+1)
        if not relevant_rules:
            print("no unary rule found for ", words[k])
            prob = g[substitute_rule]
            chart.set(k, k+1, substitute_rule[0], value=prob)
        for rule in relevant_rules:
            prob = g[rule]
            chart.set(k, k+1, rule[0], value=prob)


    examined = []
    for sub_string_length in range(2, n+1):
        # print("substring length", sub_string_length)
        # print("L", l)
        for start_idx in range(n-sub_string_length+1):
            # start_idx +=1
            # end_idx = start_idx + sub_string_length+1
            end_idx = start_idx + sub_string_length
            # print("Chart span", start_idx, end_idx)
            for mid_idx in range(start_idx +1 , end_idx):  # IDX???????
                # print("subtrees:", start_idx, mid_idx, end_idx)
                examined.append((start_idx, end_idx))
                for rule in binary_rules:
                    A, B, C = rule
                    # print(rule)
                    # g = rule[-1].astype(np.float)
                    new_prob = g[rule] * chart.get(start_idx, mid_idx, B) * chart.get(mid_idx, end_idx, C)
                    # new_prob = chart.get(start_idx, mid_idx, B) * chart.get(mid_idx, end_idx, C)
                    chart.add(start_idx, end_idx, A, value=new_prob)
            


                  
    return chart  

In [ ]:

def outside(sent, unary_rules, binary_rules, nts, nts2idx, g):
    # print("ENTERING OUTSIDE")
    words = sent.split(" ")
    # # words = sent
    # if len(words) > 5:
    #     words = words[:5]
    if len(words) <= 1:
        print("sent TOO SHORT:", sent,":")
    # print(words)
        # return g
    # words = sent
    n = len(words)
    v = len(nts)

    out_rules = {}
    for rule in binary_rules:
        # print("Rule_name", rule[:-1])
        out_rules[rule] = 0.0
    for rule in unary_rules:
        out_rules[rule] = 0.0
    counts = out_rules.copy()

    #TODO
    inside_weights = inside(words, unary_rules, binary_rules, nts,nts2idx, g)
    # print("LEAVING INSIDE")
    Z = inside_weights.get(0, n, "S")

   
        # return counts, 1
   
    out_weights = Chart(nts, nts2idx, unary_rules, binary_rules, words)

    out_weights.add(0, n, "S", value=1)
    examined = []
    for sub_string_length in reversed(range(2, n+1)): #???
        for start_idx in range(n-sub_string_length +1):
           # for start_idx in range(n-sub_string_length):
            # start_idx +=1
            end_idx = start_idx + sub_string_length
            # print("Chart span", start_idx, end_idx)

            for mid_idx in range(start_idx + 1, end_idx):  # IDX???????
                # print("subtrees:", start_idx, mid_idx, end_idx)
                examined.append((start_idx, mid_idx, end_idx))
                for rule in binary_rules:
                    # I think the first must be S
                    A, B, C= rule
                    # print("current prob", g)
                    # g = g.astype(np.float)
                    out_rules[rule] += out_weights.get(start_idx, end_idx, A) * inside_weights.get(
                        start_idx, mid_idx, B) * inside_weights.get(mid_idx, end_idx, C)
                    probB = out_weights.get(
                        start_idx, end_idx, A) * g[rule] * inside_weights.get(mid_idx, end_idx, C)
                    probC = out_weights.get(
                        start_idx, end_idx, A) * g[rule] * inside_weights.get(start_idx, mid_idx, B)
                    out_weights.add(start_idx, mid_idx, B, probB)
                    out_weights.add(mid_idx, end_idx, C, probC)

    # print("EXAMINED", examined)
    for k in range(n-1):
        relevant_rules = [x for x in unary_rules if words[k] in x]
        # print("Chart span", k, k+1)
        for rule in relevant_rules:
            A, w= rule
            # print(rule)
            out_rules[rule] += out_weights.get(k, k+1, A)

    try:
        assert Z > 0.0
    except AssertionError:
        # TODO - percolate this better
        for rule in unary_rules:
            counts[rule] = g[rule]
            # counts[rule] = 0
        for rule in binary_rules:
            counts[rule] = g[rule]
            # counts[rule] = 0
        # print("_____")
        print("NO TREE FOUND - Z:0")
        print("INSIDE")
        print(inside_weights.print())
        print("OUTSIDE")
        print(out_weights.print())
        # COUNT_Z_IS_ZERO +=1
        print(words)
        return counts, Z


    for rule in unary_rules:
        counts[rule] = (1/Z) * out_rules[rule] * g[rule]
    for rule in binary_rules:
        counts[rule] = (1/Z) *out_rules[rule] * g[rule]
        
    return counts, Z


In [ ]:
def get_list_of_rules_starting_with(A, keys):
    # keys = list(rules.keys())
    A_keys = [key for key in keys if key[0] == A]
    if len(A_keys) == 0:
        return []
    return A_keys

# Expectation Maximization


In [ ]:
def e_step(rules, sents, nts, nts2idx, G):
    # expected_counts = outside()
    unary_rules = rules[0]
    binary_rules = rules[1]

    print("calculating expected counts...")
    expected_counts_vec = []
    Zs = []
    progress_bar = tqdm(sents)
    for sent in sents:
        # print(sent.split())
        if len(sent.split()) <= 2:
            continue
        count, Z = outside(sent, unary_rules, binary_rules, nts, nts2idx, G)
        expected_counts_vec.append(count)
        Zs.append(Z)
        progress_bar.update()
    progress_bar.close()
    
    avg_likelihood = np.mean(Zs)
    Zs = [Z for Z in Zs if Z >0.0]
    avg_log_likelihood = np.log(Zs)
    var = np.std(avg_log_likelihood)
    avg_log_likelihood = np.mean(avg_log_likelihood)
    # print("avg_likelihood", avg_likelihood)
    print("avg_log_likelihood?", avg_log_likelihood)
    print("var", var)


    # likelihood = get_likelihood(expected_counts_vec, unary_rules, binary_rules, G)
    # print("new likelihood", likelihood) # I do think its log likelihood...
    return expected_counts_vec, avg_log_likelihood

    def e_step2(rules, sents, nts, nts2idx, G):
    # expected_counts = outside()
        unary_rules = rules[0]
        binary_rules = rules[1]

        print("calculating expected counts...")
        expected_counts_vec = []
        Zs = []
        progress_bar = tqdm(sents)
        for sent in sents:
            # print(sent.split())
            if len(sent.split()) <= 2:
                continue
            count, Z = outside(sent, unary_rules, binary_rules, nts, nts2idx, G)
            expected_counts_vec.append(count)
            Zs.append(Z)
            progress_bar.update()
        progress_bar.close()
        
        avg_likelihood = np.mean(Zs)
        Zs = [Z for Z in Zs if Z >0.0]
        avg_log_likelihood = np.log(Zs)
        var = np.std(avg_log_likelihood)
        avg_log_likelihood = np.mean(avg_log_likelihood)
        # print("avg_likelihood", avg_likelihood)
        # print("avg_log_likelihood?", avg_log_likelihood)
        # print("var", var)


        log_likelihood = get_likelihood(expected_counts_vec, unary_rules, binary_rules, G)
        print("likelihood", likelihood) 
        return expected_counts_vec, log_likelihood

In [ ]:

def m_step(expected_counts_vec, nts, rules):
    # print("Example exp.counts", expected_counts_vec)
    print("M Step")

    
    unary_rules = rules[0]
    binary_rules = rules[1]
    thetas = {}
    sum_occurrences_of_A = {}

    # print("building dict summing all occurrences of A")
    # print(nts)
    for nt in nts:
        un_counts = 0
        bin_counts = 0
   
        A_rules = get_list_of_rules_starting_with(nt,unary_rules)
        # print("U", len(A_rules))
        if A_rules:
            for rule in A_rules:
                un_counts += np.sum([count[rule] for count in expected_counts_vec]) 
            # print(summed_counts)
        
        A_rules = get_list_of_rules_starting_with(nt,binary_rules)
        # print("B", len(A_rules))
        if A_rules:
            for rule in A_rules:
                bin_counts += np.sum([count[rule] for count in expected_counts_vec]) 
            
        sum_occurrences_of_A[nt] = bin_counts + un_counts
        if sum_occurrences_of_A[nt] == 0.0:
            print("ZERO", nt)



    # print("Calculating unary theta rules...")
    for rule in unary_rules:
        A,w = rule
        summed_count = np.sum([count[rule] for count in expected_counts_vec]) 

        # assert sum_occurrences_of_A[A] > 0.0
        # assert summed_count <= sum_occurrences_of_A[A]
        if sum_occurrences_of_A[A] > 0.0:
            thetas[rule] =  np.exp((summed_count / sum_occurrences_of_A[A]))
            # print("sum occurrences",sum_occurrences_of_A[A] )
            # print("nonzero rule", rule, thetas[rule])
        else:
            thetas[rule] =  0.0
    # except:
        # thetas[rule] =  0.0
            
    # print("Calculating binary theta rules...")
    for rule in binary_rules:

        A,B,C = rule
        summed_count = sum([count[rule] for count in expected_counts_vec]) 
       
        # assert summed_count <= sum_occurrences_of_A[A]
        if sum_occurrences_of_A[A] > 0.0:
            thetas[rule] =  np.exp((summed_count / sum_occurrences_of_A[rule[0]]))
            # print("sum occurrences",sum_occurrences_of_A[A] )
            # print("nonzero rule",rule, thetas[rule])
        else:
            thetas[rule] =  0.0

    return thetas


In [ ]:
def EM(sents, rules, nts, nts2idx, G, max_iter=10):
    THRESHOLD = .01
    THRESHOLD = .1
    MIN_THETA = .0001
    change = 1
    avg_loglikelihoods = []
    iters = 1
    g = G
    thetas = []
    avg_likelihoods = []

    unary_rules, binary_rules = rules
   
    # rules = [unary_tuples, binary_tuples]

    while True:
        print("ITER", iters)
        # _, loglikelihoods = e_step(rules, sents)
        expected_counts_vec, avg_likelihood = e_step(rules, sents, nts, nts2idx, g)

        avg_likelihoods.append(avg_likelihood)
        if len(avg_likelihoods) > 2:
            print("DIFF", abs(avg_likelihoods[-1] - avg_likelihoods[-2]))

        if len(avg_likelihoods) > 2 and abs(avg_likelihoods[-1] - avg_likelihoods[-2]) < THRESHOLD:
            print("NO IMPROVEMENT")
            break
        elif iters >max_iter:
            break
        # E step:
        # u_thetas, b_thetas = m_step(expected_counts_vec, rules)
        thetas = m_step(expected_counts_vec, nts, rules)
        # g = thetas

        # TODO - flex g
        for rule in unary_rules:
            # these if statements are unnecesary
            if thetas[rule] < MIN_THETA:
                g[rule] = np.exp(thetas[rule])
            else:
                g[rule] = np.exp(thetas[rule])
        for rule in binary_rules:
            if thetas[rule] < MIN_THETA:
                g[rule] = np.exp(thetas[rule])
            else:
                g[rule] = np.exp(thetas[rule])

        # break
        iters += 1 
        
    print("EXITING after", iters, "iters of e_step")
    return g, thetas, avg_likelihoods[-1]



In [ ]:
def train(path_to_sents, path_to_pcfg):
    
    train_sents = load_text(path_to_sents)
    unary_rules, binary_rules, nts = load_rules_from_pcfg(path_to_pcfg)

    print(unary_rules[0])
    print(binary_rules[0])
    print(nts)
    print("len(binary_rules)", len(binary_rules))
    print("len(unary_rules)", len(unary_rules))
    print("len rules", len(unary_rules) +  len(binary_rules))

    nts2idx = {}
    for i, nt in enumerate(nts):
        nts2idx[nt] = i
    
    G = {}
    binary_tuples = []
    for rule in binary_rules:
        A, B, C, g = rule
        key = (A,B,C)
        G[key] = g.astype(np.float)
        binary_tuples.append(key)
    unary_tuples = []
    for rule in unary_rules:
        A, w, g = rule
        key = (A,w)
        G[key] = g.astype(float)
        unary_tuples.append(key)
    rules = [unary_tuples, binary_tuples]
    # print(G)


    
    g, t, final_ll = EM(train_sents, rules, nts, nts2idx, G)
    return g,t, final_ll
  



In [ ]:
def get_likelihood(expected_counts_vec, unary_rules, binary_rules, g):
    likelihood = 0 # shape 
    for count in expected_counts_vec: 
        for rule in unary_rules:
            likelihood += count[rule] * np.log(g[rule])
        for rule in binary_rules:
            likelihood += count[rule] * np.log(g[rule])
    return likelihood 

In [ ]:
# def get_likelihood(expected_counts_vec, unary_rules, binary_rules, g):
#     likelihood = 0 # shape 
#     # for count in expected_counts_vec: 
#     for rule in unary_rules:
#         count = np.sum([count[rule] for count in expected_counts_vec])
#         likelihood += count * np.log(g[rule])
#     for rule in binary_rules:
#         count = np.sum([count[rule] for count in expected_counts_vec])
#         likelihood += count * np.log(g[rule])
#     return likelihood

In [ ]:
# def parse_tree():

def test(path_to_sents, path_to_pcfg, path_to_dict):
    test_sents = load_text(path_to_sents)
    unary_rules, binary_rules, nts = load_rules_from_pcfg(path_to_pcfg)
   
    print(unary_rules[0])
    print(binary_rules[0])
    print(nts)
    print("len(binary_rules)", len(binary_rules))
    print("len(unary_rules)", len(unary_rules))

    nts2idx = {}
    for i, nt in enumerate(nts):
        nts2idx[nt] = i

    G_str = np.load(path_to_dict, allow_pickle=True)
    G_str = G_str[()]
    print(type(G_str))
    # print(G.items())
    print("G",len(G_str.items()),"R", len(unary_rules) + len(binary_rules))
    assert len(G_str.items()) == len(unary_rules) + len(binary_rules)
    G = {}
    binary_tuples = []
    for rule in binary_rules:
        A, B, C, g = rule
        key = (A,B,C)
        g = G_str[key]
        G[key] = g.astype(np.float)
        binary_tuples.append(key)
    unary_tuples = []
    for rule in unary_rules:
        A, w, g = rule
        key = (A,w)
        g = G_str[key]
        G[key] = g.astype(float)
        unary_tuples.append(key)
    rules = [unary_tuples, binary_tuples]


    expected_counts_vec, avg_likelihood = e_step2(rules, test_sents, nts, nts2idx, G)


    return expected_counts_vec, avg_likelihood




In [ ]:
def main():
    author_names = ["Alice Mabel", "Ayn Rand", "Bram Stoker", "Charles Dickens", "Emily Bronte", "Frederick Douglas", "Herman Hesse", "Herman Melville", "Jane Austen", "Leo Tolstoy", "Lewis Carroll" , "Mary Shelly Wollstonecraft", "Scott Fitzgerald", "Migel de Cervantes", "Ulysses"]
    lls = []
    for i, name in enumerate(author_names):
        print("TRAINING" , name, i)
        path_to_sents = "../make-data/train_sentences_10/training_author_idx_" + i + ".txt"
        path_to_pcfg =  "../make-data/pcfg_ug_flat_7000.txt"
        # path_to_dict= "../make-data/output_10_7000/g_model_author_idx_"+ i  + ".txt"
        # vec, ll = train(path_to_sents, path_to_pcfg, path_to_dict)
        print(ll)
        # lls.append(ll)
        # np.save(path_to_output_dir + "g_model_author_idx_" + str(i) , G)
        # np.save(path_to_output_dir + "t_model_author_idx_" + str(i) , T)


In [ ]:
author_names = ["Alice Mabel", "Ayn Rand", "Bram Stoker", "Charles Dickens", "Emily Bronte", "Frederick Douglas", "Herman Hesse", "Herman Melville", "Jane Austen", "Leo Tolstoy", "Lewis Carroll" , "Mary Shelly Wollstonecraft", "Scott Fitzgerald", "Migel de Cervantes", "James Joyce"]

# Need some file names in here
for a, author in enumerate(author_names):
    print("TRAINING" , a, author )
    
    path_to_sents = "" + str(a) + ".txt"
    path_to_pcfg =  ""
    path_to_output_dir = ""
    G, T, ll = train(path_to_sents, path_to_pcfg)
    print(ll)

    np.save(path_to_output_dir + "g_model_author_idx_" + str(a) , G)
    np.save(path_to_output_dir + "t_model_author_idx_" + str(a) , T)

In [ ]:
author_names = ["Alice Mabel", "Ayn Rand", "Bram Stoker", "Charles Dickens", "Emily Bronte", "Frederick Douglas", "Herman Hesse", "Herman Melville", "Jane Austen", "Leo Tolstoy", "Lewis Carroll" , "Mary Shelly Wollstonecraft", "Scott Fitzgerald", "Migel de Cervantes", "James Joyces"]


# Need some file names in here
lls = []
vecs = []
for a, author in enumerate(author_names):
    print("TESTING" , a, author)
    path_to_sents = '' + str(i) + ".txt"
    path_to_pcfg =  ''
    path_to_dict= " " + str(a)  + ".npy"
    vec , ll = test(path_to_sents, path_to_pcfg, path_to_dict)
    lls.append(ll)
    print(ll)

np.save("", lls)

